In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import requests
import pandas as pd
import numpy as np
from pandas.api.types import is_integer_dtype, is_float_dtype, is_string_dtype

from src.preprocess import preprocess
from src.utils import df_na_vals, np_min_chunk

In [49]:
url = 'https://500.farm/vastai-exporter/offers'

r = requests.pop(url)
r.raise_for_status()

raw  = pd.DataFrame(r.json()["offers"])
raw['timestamp'] = pd.to_datetime(r.json()['timestamp'])
print(raw.timestamp[0])
# raw.set_index('machine_id', inplace=True)
raw

2023-07-31 04:03:43.142564204+00:00


,bundle_id,bw_nvlink,compute_cap,cpu_cores,cpu_cores_effective,cpu_name,cpu_ram,credit_balance,credit_discount,credit_discount_max,...,reliability2,rentable,score,start_date,static_ip,storage_cost,total_flops,verification,verified,timestamp
0,314697833,0.0,890,256.0,256.0,AMD EPYC 7H12 64-Core Processor,515762,None,None,0.4,...,0.982502,False,16.586195,NaN,True,0.15,1438.18752,verified,True,2023-07-31 04:03:43.142564204+00:00
1,314697840,0.0,890,256.0,256.0,AMD EPYC 7662 64-Core Processor,515762,None,None,0.4,...,0.982373,False,16.618894,NaN,True,0.15,1436.22144,verified,True,2023-07-31 04:03:43.142564204+00:00
2,314525846,0.0,890,192.0,192.0,AMD EPYC 7R12 48-Core Processor,515814,None,None,0.4,...,0.991838,False,24.657660,NaN,True,0.35,1431.30624,verified,True,2023-07-31 04:03:43.142564204+00:00
3,314697828,0.0,890,256.0,256.0,AMD EPYC 7742 64-Core Processor,515841,None,None,0.4,...,0.984646,False,23.022774,NaN,True,0.15,1227.81696,verified,True,2023-07-31 04:03:43.142564204+00:00
4,314704005,0.0,890,112.0,112.0,AMD EPYC 7663 56-Core Processor,515632,None,None,0.2,...,0.994753,False,37.747514,NaN,False,0.15,1220.93568,verified,True,2023-07-31 04:03:43.142564204+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5371,313990227,0.0,750,8.0,8.0,Core™ i7 930,15987,None,None,NaN,...,0.997656,False,123.207980,NaN,True,0.05,6.63552,unverified,False,2023-07-31 04:03:43.142564204+00:00
5372,313990227,0.0,750,8.0,8.0,Core™ i7 930,15987,None,None,NaN,...,0.998317,False,118.437761,NaN,True,0.05,6.63552,unverified,False,2023-07-31 04:03:43.142564204+00:00
5373,313990241,0.0,750,8.0,8.0,Core™ i7 950,11955,None,None,NaN,...,0.998444,False,94.509620,NaN,True,0.05,4.46448,unverified,False,2023-07-31 04:03:43.142564204+00:00
5374,314716154,0.0,750,20.0,20.0,Xeon® E5-2690 v2,48090,None,None,NaN,...,0.982533,True,8.011469,1.690776e+09,True,1.50,3.81696,unverified,False,2023-07-31 04:03:43.142564204+00:00


In [45]:
raw.geolocation

0          Oklahoma, US
1          Oklahoma, US
2       Taipei City, TW
3           Taoyuan, TW
4          Oklahoma, US
             ...       
5372          Tokyo, JP
5373          Japan, JP
5374          Japan, JP
5375          Hanoi, VN
5376          Viken, NO
Name: geolocation, Length: 5377, dtype: object

In [62]:
df = np_min_chunk(raw).set_index('machine_id')


In [64]:
df.index.name

'machine_id'

In [69]:
df.id.duplicated().any()

False

In [46]:
df_na_vals(raw)

bundle_id                 ->          Missing values: 0 (0.00%)
bw_nvlink                 ->          Missing values: 0 (0.00%)
compute_cap               ->          Missing values: 0 (0.00%)
cpu_cores                 ->          Missing values: 26 (0.48%)
cpu_cores_effective       ->          Missing values: 26 (0.48%)
cpu_name                  ->          Missing values: 83 (1.54%)
cpu_ram                   ->          Missing values: 0 (0.00%)
credit_balance            ->          Missing values: 5377 (100.00%)
credit_discount           ->          Missing values: 5377 (100.00%)
credit_discount_max       ->          Missing values: 1304 (24.25%)
cuda_max_good             ->          Missing values: 0 (0.00%)
direct_port_count         ->          Missing values: 0 (0.00%)
discount_rate             ->          Missing values: 1304 (24.25%)
discounted_dph_total      ->          Missing values: 0 (0.00%)
discounted_hourly         ->          Missing values: 0 (0.00%)
disk_bw            

['credit_balance', 'credit_discount']

### 1. Country

In [32]:
country_geoloc = raw.geolocation.str.split(',').apply(lambda x: x[-1] if x else None).str.strip().replace('Sweden', 'SE')
country_geoloc.value_counts(dropna=False)

SE      1343
US      1003
CA       757
NO       281
RO       191
JP       190
FI       127
TW       104
VN        98
AR        95
RU        87
HR        80
NL        73
UA        68
ES        65
FR        61
CN        61
KR        61
CZ        53
PL        53
ZA        49
EE        49
GB        45
IL        43
TR        35
IS        34
PT        34
DE        30
SI        24
DK        22
KZ        13
IN        13
BG        10
BE         8
BR         8
LT         8
None       7
CH         7
HU         7
NZ         6
KW         5
MY         5
TT         4
MK         3
MO         3
SG         3
UZ         2
ID         2
IT         1
AT         1
Name: geolocation, dtype: int64

In [33]:
raw.loc[raw.location.isna(), 'location'] = None

AttributeError: 'DataFrame' object has no attribute 'location'

In [ ]:
country_loc = raw.location.apply(lambda x: x['country'] if x else None)

In [ ]:
raw[['geolocation', 'location', 'public_ipaddr']][country_loc != country_geoloc]

In [ ]:
""" `location` is more accurate then `geolocation`
    set country based on `location` then for missing add from `geolocation`
"""

raw['country'] = country_loc

In [ ]:
mask = raw.country.isna()
raw.loc[mask, 'country'] = country_geoloc[mask]

In [ ]:
raw.end_date.value_counts(dropna=False)

In [ ]:
def _fillna(raw: pd.DataFrame):
    """ Fill NA's:  -1 for numerical types
                    '' for string types
    """
    raw.credit_discount_max.fillna(0, inplace=True)
    raw.hosting_type.fillna(0, inplace=True)
    # raw.end_date.fillna(0, inplace=True)
    # raw.disk_bw.fillna(0, inplace=True)

    for col in raw.columns:
        dtype = raw[col].dtype
        if is_integer_dtype(dtype) or is_float_dtype(dtype):
            raw[col].fillna(-1, inplace=True)
        elif is_string_dtype(dtype):
            raw[col].fillna('', inplace=True)

### 2. Data types

In [34]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5332 entries, 11446 to 10914
Data columns (total 57 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   bundle_id             5332 non-null   int64              
 1   bw_nvlink             5332 non-null   float64            
 2   compute_cap           5332 non-null   int64              
 3   cpu_cores             5306 non-null   float64            
 4   cpu_cores_effective   5306 non-null   float64            
 5   cpu_name              5249 non-null   object             
 6   cpu_ram               5332 non-null   int64              
 7   credit_balance        0 non-null      object             
 8   credit_discount       0 non-null      object             
 9   credit_discount_max   3999 non-null   float64            
 10  cuda_max_good         5332 non-null   float64            
 11  direct_port_count     5332 non-null   int64              
 12  d

In [35]:
raw.score

machine_id
11446     14.608709
11441     16.849042
10298     24.248312
10163     27.155582
11580     22.734187
            ...    
9532     122.977727
11511     54.554066
9528      91.495069
11250      8.019109
10914      1.802470
Name: score, Length: 5332, dtype: float64